In [ ]:
## 将Dt 亚组区间转换为3Kb 大小
import pandas as pd
import sys
homo_f = sys.argv[1]
input_f = sys.argv[2]
output_f = sys.argv[3]
df_homo = pd.read_csv(homo_f, sep='\t', names=['c1', 's1', 'e1', 'c2', 's2', 'e2'])
df_inpf = pd.read_csv(input_f, sep='\t', names=['c', 's', 'e'])
df_homo_copy = df_homo.copy()
for index, row in df_homo.iterrows():
    c_dt, s_dt, e_dt = row['c2'], row['s2'], row['e2']
    print(c_dt, s_dt, e_dt)
    df_tmp = df_inpf.query("s<=@e_dt & e>=@s_dt").copy()
    print(df_tmp)
    df_tmp.reset_index(drop=True, inplace=True)
    if df_tmp.shape[0] == 1:
        df_homo_copy.loc[index, 'c3'] = df_tmp.loc[0, 'c']
        df_homo_copy.loc[index, 's3'] = df_tmp.loc[0, 's'] 
        df_homo_copy.loc[index, 'e3'] = df_tmp.loc[0, 'e']
        print(df_tmp.loc[0,:])
    else:
        for index2, row2 in df_tmp.iterrows():
            s_tmp, e_tmp = row2['s'], row2['e']
            df_tmp.loc[index2, 'length'] = min([e_tmp, e_dt]) - max([s_tmp, s_dt])
        print(df_tmp)
        df_tmp.sort_values(by=['length'], ascending=False, inplace=True)
        df_tmp.reset_index(drop=True, inplace=True)
        df_homo_copy.loc[index, 'c3'] = df_tmp.loc[0, 'c']
        df_homo_copy.loc[index, 's3'] = df_tmp.loc[0, 's'] 
        df_homo_copy.loc[index, 'e3'] = df_tmp.loc[0, 'e']
        print(df_tmp.loc[0,:])
df_homo_copy['s3'] = df_homo_copy['s3'].astype(int) 
df_homo_copy['e3'] = df_homo_copy['e3'].astype(int) 
df_homo_copy.to_csv(output_f, sep="\t", index=False)


In [9]:
## 选择染色体内的同源区域
import pandas as pd
import sys
import re
input_f = sys.argv[1]
output_f = sys.argv[2]
df = pd.read_csv(input_f, sep='\t')
pattern = r'[0-9]+'
df['at_id'] = df['c1'].str.findall(pattern)
df['dt_id'] = df['c3'].str.findall(pattern)
df_result = df.query("at_id == dt_id")
df_result.to_csv(output_f, sep='\t', index=False)


In [ ]:
## 给loop 连接的非基因区域打上enhancer 标签
import pandas as pd
df_loop = pd.read_csv("F:/TM-1_leaf_3D-genome/HiC/loop/gene_loop_new/all_nogene_genes_loop.txt", sep='\t', names=['c', 's', 'e', 'geneid', 'fpkm'])
df_enhancer = pd.read_csv("F:/TM-1_leaf_3D-genome/HiC/loop/gene_loop_new/enhancer/all_At_Dt_homo_enhaner.txt", sep='\t', names=['c', 's', 'e'])
print(df_loop)
print(df_enhancer)
df_result = pd.merge(df_loop, df_enhancer, how='inner', on=['c', 's', 'e'])
print(df_result)


In [ ]:
## 分析loop 所能包含的基因
import pandas as pd
import sys
gene_f = sys.argv[1]
loop_f = sys.argv[2]
output_f = sys.argv[3]
df_gene = pd.read_csv(gene_f, sep='\t')
df_loop = pd.read_csv(loop_f, sep='\t')
df_result = pd.DataFrame(columns = ['c1', 's1', 'e1', 'gid1', 'c2', 's2', 'e2', 'gid2', 'tags'])
for index, row in df_loop.interrows():
    c1,s1,e1,c2,s2,e2 = row['c1'], row['s1'], row['e1'], row['c2'], row['s2'], row['e2']
    df_gene_one_tmp = df_gene.query("@s1<=e & @e1>=s").copy()
    df_gene_two_tmp = df_gene.query("@s2<=e & @e2>=s").copy()
    ## GG 注释
    if df_gene_one_tmp.shape[0]>0 & df_gene_two_tmp.shape[0]>0:
        gid1, gid2 = df_gene_one_tmp['geneid'].tolist(), df_gene_two_tmp['geneid'].tolist()
        for i in gid1:
            for j in gid2:
                df_result.loc[df_result.shape[0], :] = [c1, s1, e1, i, c2, s2, e2, j, 'GG']
    ## GN 注释
    elif df_gene_one_tmp.shape[0]>0 & df_gene_two_tmp.shape[0]==0:
        gid1 = df_gene_one_tmp['geneid'].tolist()
        for i in gid1:
            df_result.loc[df_result.shape[0], :] = [c1, s1, e1, i, c2, s2, e2, 'nogene', 'GN']
    ## NG 注释
    elif df_gene_one_tmp.shape[0]==0 & df_gene_two_tmp.shape[0]>0:
        gid2 = df_gene_two_tmp['geneid'].tolist()
        for j in gid2:
            df_result.loc[df_result.shape[0], :] = [c1, s1, e1, 'nogene', c2, s2, e2, j, 'NG']
    ## NN 注释
    else:
        df_result.loc[df_result.shape[0], :] = [c1, s1, e1, 'nogene', c2, s2, e2, 'nogene', 'NN']
df_result.to_csv(output_f, sep='\t', index=False)

In [ ]:
## 分析同源enhancer数目

import pandas as pd
import sys

at_f = sys.argv[1]
dt_f = sys.argv[2]
at_dt_f = sys.argv[3]
output_f = sys.argv[4]

df_at = pd.read_csv(at_f, sep='\t', names=['c1', 's1', 'e1'])
df_dt = pd.read_csv(dt_f, sep='\t', names=['c2', 's2', 'e2'])
df_at_dt = pd.read_csv(at_dt_f, sep='\t')

df_one = pd.merge(df_at_dt, df_at, how='inner', on=['c1', 's1', 'e1'])
df_two = pd.merge(df_one, df_dt, how='inner', on=['c2', 's2', 'e2'])

df_two.to_csv(output_f, sep='\t', index=False)


In [ ]:
## 分析同源基因和同源enhancer 之间的联系
import pandas as pd
import sys
input_f1 = sys.argv[1]
input_f2 = sys.argv[2]
output_f = sys.argv[3]
df1 = pd.read_csv(input_f1, sep="\t", names=['c1', 's1', 'e1', 't1', 'c2', 's2', 'e2', 'geneid' ])
df2 = pd.read_csv(input_f2, sep="\t", names=['c', 's', 'e', 'geneid'])
df_result = pd.merge(df1, df2, how='inner', on=['geneid'])
df_result.to_csv(output_f, sep="\t", index=False, columns=['c1', 's1', 'e1', 't1', 'c2', 's2', 'e2', 'geneid'], header=False)

In [ ]:
## 分析enhancer - gene At-Dt亚组相同的loop
import pandas as pd
import sys
GN_loop = sys.argv[1]
HNs = sys.argv[2]
HGs = sys.argv[3]
output_f = sys.argv[4]
df_gn = pd.read_csv(GN_loop, sep="\t", names=['N', 'G'])
df_hn = pd.read_csv(HNs, sep='\t', names=['N', 'N1'])
df_hg = pd.reac_csv(HGs, sep='\t', naems=['G', 'G1'])

df_hn2 = df_hn[['N1', 'N']].copy()
df_hn2.columns =['N', 'N1']
df_hg2 = df_hg[['G1', 'G']].copy()
df_hg2.columns =['G', 'G1']

df_hn = df_hn.append(df_hn2, ignore_index=True)
df_hg = df_hg.append(df_hg2, ignore_index=True)

df_one = pd.merge(df_gn, df_hn, how='inner', on=['N'])
df_two = pd.merge(df_one, df_hg, how='inner', on=['G'])
df_two['tags'] = [1]*df_two.shape[0]
df_result = df_two.grouby(['N', 'N1', 'G', 'G1'], as_index-False)['tags'].sum()

In [3]:
import pandas as pd
df = pd.DataFrame({'a':[1,2,3], 'b':[3,4,5]})
df2 = df[['b','a']]
print(df2)

   b  a
0  3  1
1  4  2
2  5  3
